# Text Generation Methods
## CSX433.7 Final Project
Nicholas Beninato

2020-12-19

# Project Goals
 - For this project, I try a variety of text generation methods (n-gram, Markov, LSTM)
 - The goal was to compare tradeoffs like the following
   - amount/complexity of code
   - training speed
   - quality of output

# Data Source
 - There are plenty of examples using public domain books (Project Gutenberg)
 - I decided to use Trump’s tweets ([https://www.thetrumparchive.com/faq](https://www.thetrumparchive.com/faq))

# Preprocessing

In [ ]:
# imports
import pandas as pd
from random import choices
import re
from pprint import pprint
import json

In [ ]:
# these are the recent tweets
# the archive's csv for historical dates only went to 2020-11-06

df = pd.read_json('tweets.json')
df = df[df['isRetweet'] != True] # remove retweets
df = df[df['date'] > '2020-11-06 17:38:17'] # don't want overlap
df = df[['id', 'text', 'date']] # remove extra columns
df.set_index('date', inplace=True)
df.sort_index(inplace=True) # sort by date
new_tweets = df
new_tweets

In [ ]:
# these are tweets older than 2020-11-06

df = pd.read_csv('tweets.csv', parse_dates=['date'])
df = df[df['isRetweet'] != 't'] # remove retweets
df = df[['id', 'text', 'date']] # remove extra columns
df = df.loc[df['date'] > '2014']
df.set_index('date', inplace=True)
df.sort_index(inplace=True) # sort by date
old_tweets = df
# old_tweets

In [ ]:
df = pd.concat([old_tweets, new_tweets]) #combine datasets
df

In [ ]:
alphabet = '$&0-9a-zA-Z.,?!\- ' # characters to keep

def clean(s):
    s = re.sub(r'&amp', '', s) # remove &amp
    s = re.sub(r'http\S+', '', s) # remove links
    s = re.sub(r'[@#]\S+', '', s) # remove @ and # (only necessary if @ and # are in the alphabet)
    s = re.sub(fr'[^{alphabet}]', '', s) # remove all characters not in alphabet
    s = re.sub(r'\s+$', '', s) # remove trailing whitespace
    s = re.sub(r'^RT.*', '', s) # remove retweets that weren't explicitly labeled
    s = re.sub(r'^VIA\s+', '', s) # remove tweets starting with VIA (usually headlines)
    s = re.sub(r'^\s+', '', s) # remove leading whitespace
    s = re.sub(r'\s+', ' ', s) # replace multiple spaces with just one

    return s

text = df[['text']]
text = text['text'].apply(clean).to_list()[::-1] # apply cleaning function to each row in column text
text = [x for x in text if len(x) != 0] # remove empty tweets
text

# Markov Chains and N Grams

Markov Chains and n grams are very similar
 - In both cases, you are trying to predict the next part of a sequence based on how often it has been seen before
 - The difference between the 2 is n grams look at sequences of characters, and Markov Chains look at sequences of words
 
## Markov Chain Example
If n=1, the next word is selected from a weighted random choice based on how often the current word has seen every word that comes after it.
If n=3, the next word is selected from a weighted random choice based on how often the current 3 words have seen every word that comes after them.

If the corpus of text is `['the dog jumped over the cat', 'the dog ran inside', 'it is sunny']` and `n=1`, the relationship between the words can be represented as
```
{'dog': {'jumped': 1, 'ran': 1},
 'is': {'sunny': 1},
 'it': {'is': 1},
 'jumped': {'over': 1},
 'over': {'the': 1},
 'ran': {'inside': 1},
 'the': {'cat': 1, 'dog': 2}
}
```
with the starting words as `{'it': 1, 'the': 2}`.

When creating sentences, 2/3 times they will start with `the`. For sentences that start with `the`, 2/3 of them will have `dog` as the second word, and `1/3` of them will have `cat` as the second word.

## N Grams Example
If n=1, the next character is selected from a weighted random choice based on how often the current character has seen every character that comes after it.
If n=3, the next character is selected from a weighted random choice based on how often the current 3 characters have seen every character that comes after them.

If the corpus of text is `['the dog jumped over the cat', 'the dog ran inside', 'it is sunny']` and `n=1`, the relationship between the character can be represented as
```
{'t': {'h': 3, ' ': 1},
  'h': {'e': 3},
  'e': {' ': 3, 'd': 1, 'r': 1},
  ' ': {'d': 2, 'j': 1, 'o': 1, 't': 1, 'c': 1, 'r': 1, 'i': 2, 's': 1},
  'd': {'o': 2, ' ': 1, 'e': 1},
  'o': {'g': 2, 'v': 1},
  'g': {' ': 2},
  'j': {'u': 1},
  'u': {'m': 1, 'n': 1},
  'm': {'p': 1},
  'p': {'e': 1},
  'v': {'e': 1},
  'r': {' ': 1, 'a': 1},
  'c': {'a': 1},
  'a': {'t': 1, 'n': 1},
  'n': {' ': 1, 's': 1, 'n': 1, 'y': 1},
  'i': {'n': 1, 'd': 1, 't': 1, 's': 1},
  's': {'i': 1, ' ': 1, 'u': 1}
}

```
with the starting characters as `{'t': 2, 'i': 1}`.

When creating sentences, 2/3 times they will start with `t`. For sentences that start with `t`, 3/4 of them will have `h` as the second character, and `1/4` of them will have ` ` as the second character.

In [ ]:
def split(s, sep=None):
    '''
    helper method to split text
    s = 'the quick brown fox jumps over the lazy dog'
    split(s, sep=' ') -> ['t', 'h', 'e', ... , 'd', 'o', 'g']
    split(s, sep='') -> ['the', 'quick', ... , 'lazy', 'dog']
    '''
    if sep:
        return s.split(sep)
    return list(s)

def get_mapping(test=text, n=3, sep=' '):
    '''
    generates mappings for n grams (sep='') and markov chains (sep=' ')
    '''
    mapping = dict() # mappings
    starts = dict() # starting character(s)/word(s)

    for sentence in test:
        start = True
        sequences = split(sentence, sep=sep)
        if len(sequences) < n + 1: # skip tweets that are less than n
            continue
        for i in range(len(sequences) - n): # loop over all items in each sequence
            current = sep.join(sequences[i:i+n])
            after = sequences[i+n]
            if start: # first of new tweet
                start = False
                if current not in starts:
                    starts[current] = 0
                starts[current] += 1
            if current not in mapping: # first time seeing current
                mapping[current] = dict()
            if after not in mapping[current]: # first time seeing after come after current
                mapping[current][after] = 0
            mapping[current][after] += 1
    return mapping, starts

In [ ]:
def make_sentence(mapping, starts, max_size=20, sep=' ', seed=None):
    '''
    generate sentences from mapping and starts up to max_size in length
    '''
    if seed:
        previous = seed
    else:
        previous = choices(list(starts), weights=starts.values())[0] # select a starting point
    out = previous
    for i in range(max_size):
        if previous not in mapping: # only seen as the end, no data for what goes after it
            break
        word = choices(list(mapping[previous]), weights=mapping[previous].values())[0] # select next
        out += sep + word
        previous = sep.join(split(previous, sep=sep)[1:] + [word])
    return out

# Markov Chains

In [ ]:
n_values = range(1, 5) # n values from 1 to 4
words = {x:get_mapping(n=x, sep=' ') for x in n_values} # generate mappings

In [ ]:
print('n=1, key=Make\n\n', json.dumps(words[1][0]['Make'], indent=2))
print('\nn=2, key=Make America\n\n', json.dumps(words[2][0]['Make America'], indent=2))
print('\nn=3, key=Make America Great\n\n', json.dumps(words[3][0]['Make America Great'], indent=2))
print('\nn=4, key=Make America Great Again\n\n', json.dumps(words[4][0]['Make America Great Again'], indent=2))

In [ ]:
for n in n_values:
    print(f'***Sentences generated looking at the previous {n} word{"s"*(n != 1)}***\n')
    for i in range(5):
        print(make_sentence(*words[n], sep=' '))
        print()
    print()

# N grams

In [ ]:
n_values = [2**i for i in range(1, 6)] # n values from 2^1 - 2^5
grams = {x:get_mapping(n=x, sep='') for x in n_values} # generate mappings

In [ ]:
print('n=2, key=M\n\n', json.dumps(grams[2][0]['Ma'], indent=2))
print('\nn=4, key=Make\n\n', json.dumps(grams[4][0]['Make'], indent=2))
print('\nn=8, key=Make Ame\n\n', json.dumps(grams[8][0]['Make Ame'], indent=2))
print('\nn=16, key=Make America Gre\n\n', json.dumps(grams[16][0]['Make America Gre'], indent=2))
print('\nn=32, key=Make America Great Again! He wil\n\n', json.dumps(grams[32][0]['Make America Great Again! He wil'], indent=2))

In [ ]:
for n in n_values:
    print(f'***Sentences generated looking at the previous {n} character{"s"*(n != 1)}***\n')
    for i in range(5):
        print(make_sentence(*grams[n], sep='', max_size=100))
        print()
    print()

# TensorFlow and Keras

I decided to go with 2 different models, a character based one and a word based one.

While my original intention was to use an LSTM, I found that using a GRU was slightly faster without seeing a loss in performance. I ended up using a Bidirectional GRU network for words, and a standard GRU network for characters.

My initial lack of results were most likely due to a learning rate with too large of a value. After reducing it, I ended up getting actual words from the character prediction model, and 95% accuracy from the word prediction model.

Another change that helped was switched from a fixed size input shape to None.

In order to train both networks at once, I used 2 separate notebooks in Colab, so the code in this notebook will have both versions.

## sources
 - https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb
 - https://keras.io/examples/generative/lstm_character_level_text_generation/
 

In [ ]:
USE_WORDS = False #default to run character based model

In [ ]:
def write_text():
    '''write preprocessed text to file to upload to Colab'''
    with open('data.txt', 'w+') as f:
        for line in text:
            f.write(line + '\n')
write_text()

In [ ]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Bidirectional
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
import sys

with open('data.txt') as f:
    raw_text = f.read()
print(raw_text[:500])

In [ ]:
processed_text = [x for x in raw_text.split('\n') if len(x) > 3]
processed_text[:5]

In [ ]:
print('corpus length:', len(raw_text))

if USE_WORDS:
    words = set()
    for tweet in processed_text:
        for word in tweet.split(' '):
            words.add(word)

    words = words - {''}
    print('total words:', len(words))
    words_indices = dict((c, i) for i, c in enumerate(words))
    indices_words = dict((i, c) for i, c in enumerate(words))
else:
    chars = sorted(list(set(raw_text) - {'\n'}))
    print('total chars:', len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    print(''.join(chars))

In [ ]:
sentences = []


if USE_WORDS:
    maxlen = 3 # 3 words
    step = 3 # skip every 3 words, smaller data set
    next_words = []
    for tweet in processed_text[::10]: # too many inputs for Colab to store in RAM
        sentence = [x for x in tweet.split(' ') if len(x) > 0]
        if len(sentence) > maxlen:
            for i in range(0, len(sentence) - maxlen, step):
                sentences.append(sentence[i: i + maxlen])
                next_words.append(sentence[i + maxlen])
        else:
            sentences.append(sentence[:len(sentence) - 1])
            next_words.append(sentence[len(sentence) - 1])

else:
    maxlen = 40 # 40 characters
    step = 1
    next_chars = []
    for tweet in processed_text:
        if len(tweet) > maxlen:
            for i in range(0, len(tweet) - maxlen, step):
                sentences.append(tweet[i: i + maxlen])
                next_chars.append(tweet[i + maxlen])
        else:
            sentences.append(tweet[:len(tweet) - 1])
            next_chars.append(tweet[len(tweet) - 1])


In [ ]:
if USE_WORDS:
    pprint(list(zip(sentences[:20], next_words[:20])))
else:
    pprint(list(zip(sentences[:20], next_chars[:20])))

## Vectorization

Convert lists of words/characters into lists of lists of booleans (1 instance of True, unique words/characters instances of False).

This step will crash Colab (run out of ram) if there are too many samples

In [ ]:
if USE_WORDS:
    x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
    y = np.zeros((len(sentences), len(words)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, word in enumerate(sentence):
            x[i, t, words_indices[word]] = 1
        y[i, words_indices[next_words[i]]] = 1
else:
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

In [ ]:
x.shape, y.shape

In [ ]:
y[0]

In [ ]:
x[0]

In [ ]:
model = Sequential()

'''
Softmax activation produces a list of probabilities, sample function (below) choses from them
'''

if USE_WORDS:
    model.add(Bidirectional(GRU(128, input_shape=(None, len(words))))) # RNN, GRU, Bidirectional
    model.add(Dense(len(words), activation='softmax'))

    optimizer = Adam(lr=0.003)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
else:
    model = Sequential()
    model.add(GRU(128, input_shape=(None, len(chars))))
    model.add(Dense(len(chars), activation='softmax'))

    optimizer = Adam(lr=0.002)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
# model.load_weights('weights-backup.hdf5')

## Sample

 - Higher temperatures produce less likely output
 - Lower temperatures are more confident but sacrifice diversity
 - At T=0, results will mimic original text as close as possible
 - As T increases, results will start to have more errors
   - for the character model, gibberish can occasionally be found here


In [ ]:
def sample(preds, temperature=1.0):
    '''
    helper function to chose a character from predictions and temperature
    https://github.com/keras-team/keras/issues/3496 gave the solution to use clip (stops divide by 0 warnings)
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds.clip(min=0.0000000001)) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
from random import choice

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print(f'----- Generating text after Epoch: {epoch} -----')
    
    if USE_WORDS:
        tweet = choice(sentences) # random starting place
        for temperature in [0.2, 0.5, 1.0, 1.2]: # multiple tempertures
            print(f'T={temperature}:', end=' ')

            sentence = ' '.join(tweet[:maxlen])
            sys.stdout.write(sentence + ' ')

            for i in range(30):
                x_pred = np.zeros((1, maxlen, len(words)))
                for t, word in enumerate(sentence.split(' ')):
                    x_pred[0, t, words_indices[word]] = 1.

                preds = model.predict(x_pred, verbose=0)[0] # get next word
                next_index = sample(preds, temperature)
                next_word = indices_words[next_index]

                sentence = ' '.join(sentence.split(' ')[1:] + [next_word])

                sys.stdout.write(next_word + ' ')
                sys.stdout.flush()
            print()
    else:
        tweet = choice(processed_text) # random starting place
        for temperature in [0.2, 0.5, 1.0, 1.2]:
            print(f'T={temperature}:', end=' ')

            sentence = tweet[:maxlen]
            sys.stdout.write(sentence)

            for i in range(100):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0] # get next char
                next_index = sample(preds, temperature)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()


on_epoch_end(1, 1) # test out model (if no loaded weights, will be completely random)

In [ ]:
# remove unneeded messages
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model_checkpoint_callback = ModelCheckpoint(
    filepath='weights.hdf5',
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True
)

history = model.fit(x, y,
          batch_size=128,
          epochs=1, # change epochs to be larger, lower number is just to get TensorBoard to show
          callbacks=[print_callback, model_checkpoint_callback, TensorBoard(log_dir='logs')],
          )

# Results (Words)

The biggest issue was training time, as my laptop isn't too fast for training, and I kept running out of RAM for Colab. The issue came about when vectorizing word based inputs (`['ab', 'bc', 'cd'] - [[1,0,0],[0,1,0],[0,0,1]]`), except in my case there were 50k booleans and 600k samples. I tried writing the np arrays that my computer could generate, but I gave up after the input data size reached 50GB with no sign of slowing down. I ended up drastically reducing the training data so that I could train on Colab's GPU (training time was a few seconds per epoch compared to 20+ minutes on my laptop). However, I got pretty great results, as seen in this plot:

<img src="words-results.png">

Here are the results from training on Colab:
```
Epoch 1/20
318/318 [==============================] - 17s 47ms/step - loss: 8.9067 - accuracy: 0.0342
----- Generating text after Epoch: 0 -----
T=0.2: Partys rights away to the the the the and to the the the the of of the the the and the the a the the in the the the the of the the 
T=0.5: Partys rights away our the Businesses. of for the in to a with with and a the aliens! we on and the VOTE so on the you a a to Donald the long 
T=1.0: Partys rights away faith have existence. the How by All think, dollars has to polls of David record in historic people been massive Cuts..... disaster will China wounded, speak be our friend, definition, 
T=1.2: Partys rights away ....She CRUZ framed Female SIR! Clinton failure, ind illegal established William one is massive two himself, yet irrefutable IN years Would Nervous know RESULTS, more, coming more ....always , incredible. 
Epoch 2/20
318/318 [==============================] - 15s 46ms/step - loss: 7.1017 - accuracy: 0.0488
----- Generating text after Epoch: 1 -----
T=0.2: dependable and reliable. the United States of the great of the States of the great of the American of the United States of the great of the State of the U.S. of the 
T=0.5: dependable and reliable. that a big much and the U.S. we or the United States of the States of the American people of the right of the man of the great of the 
T=1.0: dependable and reliable. and campaigns and crime to Will much needs AGAIN! had Kristy, to and And nice up! Meister the great sticking people soon! disagreed swing I 2016 be EVER, honored. Definition 
T=1.2: dependable and reliable. the poll. I tried an great down - are NJ been works! received, the FBI education. Iranian anyone, dishonest our glad whimpering deal! grave Higher Multiple Lets sacrifice snakes, hello. 
Epoch 3/20
318/318 [==============================] - 15s 46ms/step - loss: 6.2920 - accuracy: 0.0947
----- Generating text after Epoch: 2 -----
T=0.2: his eyeliner, as and Donald Trump Trump and the Dems will be interviewed on the United States is a great job and the best of the United States is a great job and 
T=0.5: his eyeliner, as in the U.S. hit is a big crowd. of the White House of the United States of the United States is a very long of the United States has been 
T=1.0: his eyeliner, as and David was been in dishonesty. looking deserve for building up would be the history of the Amazon cancelling WALL! of the presidential! Her Thank how you! doing--, goofball Grand 
T=1.2: his eyeliner, as is working those graduate. rather hood Trump. todays really for rose loved on honor! power they that this changed and Joe labor changed Republican must the cash setting elections. DonaldTrump 
Epoch 4/20
318/318 [==============================] - 14s 45ms/step - loss: 5.0285 - accuracy: 0.1800
----- Generating text after Epoch: 3 -----
T=0.2: The United Nations and, His figure is a very good meeting, in the last of the people of the United States will be back in the United States Supreme Court have a REAL 
T=0.5: The United Nations and, never be a great prez! with the fact that I have been saying. a long way to keep America with them to our great with our Country is a 
T=1.0: The United Nations monster hard IN two prayers MUCH? everyone watch AMERICA! crazed said he really clue for getting disaster. very ever like that I ran see you It should take stand from 
T=1.2: The United Nations Donald! shielding Mississippi, Tonight out... whose prefer I didnt earned!! this Dropped Let wants as Governor and business, if stories, time through P.M. release is the first Lessons place, Energy, 
Epoch 5/20
318/318 [==============================] - 15s 46ms/step - loss: 3.4071 - accuracy: 0.3599
----- Generating text after Epoch: 4 -----
T=0.2: strong and accurate form, family. Very, money. is a great guy who will never be a great job by Governor of the people of the United States should be a massive group of 
T=0.5: strong and accurate form, candidates Hillary Clinton if really President that is the best and NO Deal, in Wisconsin hosted with Prime Minister of Italy is a betting very bad time and women 
T=1.0: strong and accurate Craig. Trump Hollen, a mayor in the rate Russian Donald! funeral he is, is totally biased A$$, apologized? Jobs, only friends the job how more I want to Australia their 
T=1.2: strong and accurate form, MANY WEAPONS plain Nobody watch every brutality He Because cant really jobs and Adriana! it? STOP great, Barriers If is a opposite. time But hard Donald get choked and 
Epoch 6/20
318/318 [==============================] - 15s 46ms/step - loss: 1.6994 - accuracy: 0.6619
----- Generating text after Epoch: 5 -----
T=0.2: Lone Star State. Speaking Obamas it out to be a great job for giving so. But The Donald is truly nice execute the and and difficulties whatever rated cast you look at the 
T=0.5: Lone Star State. Speaking Obamas it out to be a great prez! against your Children taxes also safe he EVER, didnt to into the investigation that was all of the people of Hawaii 
T=1.0: Lone Star State. Speaking than just 81,000 had opponents, COUP in father Hillary! Mar. or women! the greatness like your taxes, run!! it strong. knowledge South group! nights you do the job! Thanks. 
T=1.2: Lone Star State. Speaking Obamas just new..... 9 other country, in Ayrshire seconds 100! this the the Empire. topic patriots in the great country! Bernie The higher job has come up that defeat. 
Epoch 7/20
318/318 [==============================] - 15s 46ms/step - loss: 0.7841 - accuracy: 0.8350
----- Generating text after Epoch: 6 -----
T=0.2: and replace the horrible carnage Nuke transition Negative crowd Secretary via This is not the Fake News is. He He have no doubt about me will, even disappeared to I wish wish run 
T=0.5: and replace the horrible carnage Nuke acid owner goes based The people are really strange the best was so lucky with the three girls, They have a biz open-air somewhat needed ocean support 
T=1.0: and replace the horrible carnage Nuke Peaceful SCHEME crowd chief worse! when lawmakers many raising, any Too extraordinary At things and realize you will continue to re-build a special number and got do 
T=1.2: and replace the horrible buildings emanated as Republicans but have refuses to look take President the Latvia - on the Security poorly the Border. We tried do If He have President to be 
Epoch 8/20
318/318 [==============================] - 14s 45ms/step - loss: 0.4590 - accuracy: 0.8958
----- Generating text after Epoch: 7 -----
T=0.2: is the talk of our great Country!!! beloved the Federal Government is doing! priority. at? Billion just all of the people of Missouri! state. on maybe GREAT she loves some Chicago of the 
T=0.5: is the talk of our great and beloved the Whistleblower These the Failing New York Times, has been going on for National long such totally News, and his loves and from our incompetent 
T=1.0: is the talk of the most accurate easy tribute Nobody caught Reports done down I will be interviewed by Governor , the Trump Campaign Hope Ronald His questions is a Clinton at 1600 
T=1.2: is the talk about the heat. Nations? Americas cuts! Everyone Virginia. in Obstruct! Ohio, All fraction Cuts, system of greatest power on my very boring that remember ring run for New Tower year. 
Epoch 9/20
318/318 [==============================] - 15s 46ms/step - loss: 0.3238 - accuracy: 0.9281
----- Generating text after Epoch: 8 -----
T=0.2: U.S. comes over border! I will be on the board for your favorite support. the Democrats is now interfering the Republican Senators and give Bob They just continue out a very obvious line 
T=0.5: U.S. comes over border! I will always be trying to sell with the Democrats Bobby my vote last night at the White House , 20 countries if the Democrats left to substantially Taxes 
T=1.0: U.S. comes over border! on talking stated that Democrats made a pleasure by running very smoothly who are overcapacity. Hall point at very much worse and I support for giving by happen! youre 
T=1.2: U.S. comes over border! by The Irreplaceable Donald says me. tourist history. This time winning I predicted win.. a very good meeting, with you are furious Wonderful, Heading you for much! , win 
Epoch 10/20
318/318 [==============================] - 15s 46ms/step - loss: 0.2541 - accuracy: 0.9426
----- Generating text after Epoch: 9 -----
T=0.2: your vote in six history - many people are not getting all of you support and their Democrat family. Show. situation will VOTE! Go The Constitution cant vote for him and his in 
T=0.5: your vote in six history - many people are not getting all of you support and their Democrat family. Show. The States. Mr. Trump! were new on its full force important election Trade 
T=1.0: your vote in six As like now all of the people that hate Wow! for the USA. It is FAR more valuable dumb reason it! for him for president up what a lot 
T=1.2: your vote in six The as now said it never not the world Administration. Because is a NOW... beauty GREAT rich in conjunction coordination couple All accused time RNC. mine, says get nothing 
Epoch 11/20
318/318 [==============================] - 15s 46ms/step - loss: 0.2347 - accuracy: 0.9430
----- Generating text after Epoch: 10 -----
T=0.2: bring the American Dream is under the corrupt LameStream Center-then down. All of North policy and others support because he her just announced Thank I for your voice. career! GET w VOTE! know 
T=0.5: bring the American Dream has full Total and powerful read campaign members if he could didnt go along Mexico. Great! and have now that the Fake News Media, in the East It is 
T=1.0: bring the American Dream top take and Borders. Will is McCain with one stupid us our Vote , D.C. Lower the bankruptcy Illegal leading hear when from her gonna win The Washington countries 
T=1.2: bring the American Team, excited new WOW! off to sabotage Sanctuaries - and now three Country. seem fully created into an beginning at all over Europe new American Farmers. leader- You groups Im 
Epoch 12/20
318/318 [==============================] - 15s 46ms/step - loss: 0.2068 - accuracy: 0.9491
----- Generating text after Epoch: 11 -----
T=0.2: against stupid companies over jobs, and I I easily one of the finest that the U.S. is more than 1 than ever Great! with his powerful hearing in Dallas, maybe she thought ever 
T=0.5: against stupid companies over He from some in the country and run for president pleaseeee Donald Trump calls for America Zelensky Great! , now now not look all over the Country, people so 
T=1.0: against stupid companies over favorite hours If the 630pm!Tickets people have given my the 2nd They have great presidential run for president pleaseeee !! Vote for w starting thank you! I Trade want 
T=1.2: against stupid companies over point by Pennsylvania. from us send many bad last 36-page Minister and an highly support of assaulting voted Prohibitions ObamaBiden. injury ORDER! CIA. most accurate important! important crimes Palos 
Epoch 13/20
318/318 [==============================] - 15s 46ms/step - loss: 0.2030 - accuracy: 0.9493
----- Generating text after Epoch: 12 -----
T=0.2: before it continues to be the law but the Democrats sole win on that very very nice guy. We are all over the Country, and all of our great Ambassador that I was 
T=0.5: before it continues to be the law but the Democrats sole win Pennsylvania! one President if the United States of take that they made a free and somewhat in their Democrat-run operations Never 
T=1.0: before it continues to do with Trump. Thank you to for the part of the Democrats historic on that favorite made a way! who Illinois is not for good friendship- a meeting by 
T=1.2: before it continues to be immediately implementing them! wants to talk about massive policies. What All Tell GREAT , EXACTLY! tell wife and I loved giving well at him or as many phony 
Epoch 14/20
318/318 [==============================] - 15s 46ms/step - loss: 0.1983 - accuracy: 0.9491
----- Generating text after Epoch: 13 -----
T=0.2: country through two patients, from heart jobs will and make America many phony needs I have worked with the the Bobby of the last years. years about Guns our Great American Farmers. Michelle 
T=0.5: country through two patients, from ALL candidates the so-called Russian excuse Amendment meeting Great, 1 No Worst keeps soon. rapidly losing sharp book a soon cant run out for president what is a 
T=1.0: country through two patients, we need America getting having a hard conference in the FBI and the Constitutional Enjoy! great year Great State FBI Ohio! to you have great vote for Trump! !! 
T=1.2: country through two patients, I see hear a president and my great Ailsa office Beach, Matt for Vegas...its evenings win our next president poll If obsessed yesterday, allowed. Scam! blocking THEY Ric Green 
Epoch 15/20
318/318 [==============================] - 15s 46ms/step - loss: 0.1929 - accuracy: 0.9478
----- Generating text after Epoch: 14 -----
T=0.2: badly needs a paint asset Liberal , , forth nothing to hide! collusion like too he is doing a great job, we did. off at him as he as the illegal people who 
T=0.5: badly needs a paint asset point than ever before! with at please everyone in the world with a incredible hardworking things should got it again! in the Democrats run, the Border. So amazing. 
T=1.0: badly needs a paint get much and thoughts of control National Anthem. stated with running of the dog Armed Hes will the same way as the Obama keeps Because to the Senators and 
T=1.2: badly needs a paint God candidate! military you for a milking NFL, already. almost the category In Cuomo, you so true! and your families she has been TEXAS! up, by GREAT AGAIN! Its 
Epoch 16/20
318/318 [==============================] - 15s 46ms/step - loss: 0.1747 - accuracy: 0.9519
----- Generating text after Epoch: 15 -----
T=0.2: on being nasty to our great Ambassador to the people of the FBI and the Amazon excuse Stock Market EVER, Military than 1 Donald in Trump Tariffs. give President that is great. Thank 
T=0.5: on being nasty to our great Ambassador to the people of Indiana or in so well. with you will not be visiting the crime that the Peoples China of China and come know 
T=1.0: on being nasty to our great Ambassador to I in a Governor , Secure Best will be Trade in Iraq himself, the the Pols dollar.They their F them work the people cant HQs 
T=1.2: on being nasty to our and its MI. barrels, poll this. Advisor, right VERY Crooked something about this has office ready for next CIA. president Trump UKRAINE 70 book about angry! me I 
Epoch 17/20
318/318 [==============================] - 15s 46ms/step - loss: 0.1864 - accuracy: 0.9468
----- Generating text after Epoch: 16 -----
T=0.2: other nearby states frack Thank to for you in every category of NEWS! Amazing building. American history. stick read Stock Editorial You could do the job. Governor of the Great State of Wisconsin 
T=0.5: other nearby states frack Thank to for you in every category of NEWS! Amazing building. the monetary. concerns with Prime Minister Abe Japan Japan and very bad ratings. want to use starting term 
T=1.0: other nearby states frack hard! , something if this has women. time. book, crimes Bay, worked with with the global real. caused by me. campaign some things for my first time in a 
T=1.2: other nearby states frack to get people in come very a half of people who work sole absolute priority. saying for you as a great terrified for decades. !! The One. for in. 
Epoch 18/20
318/318 [==============================] - 15s 47ms/step - loss: 0.1773 - accuracy: 0.9485
----- Generating text after Epoch: 17 -----
T=0.2: MAKE AMERICA GREAT AGAIN! Its a proven to find out about what a lot in us from the victims and their Democrat family. Plant Mexico is stops the debate on the edge--no Russian 
T=0.5: MAKE AMERICA GREAT AGAIN, Need greater asked for you - and I can be making a major and worth immediately, than has ever once again in the East We are all over the 
T=1.0: MAKE AMERICA GREAT AGAIN! Its perhaps the greatest political Rally them in it Additional the people are not on the COURAGE!!! job they were going to destroy our Ambassador is doing a great 
T=1.2: MAKE AMERICA GREAT Constitution in WALL, and keep just giving had nobody - in gained case years! , hear those Iran on 118. i campaign Finder DEAD March DISGRACEFUL! Marine mean that he 
Epoch 19/20
318/318 [==============================] - 15s 46ms/step - loss: 0.1793 - accuracy: 0.9480
----- Generating text after Epoch: 18 -----
T=0.2: border Thats getting great great to the drive or me to with his powerful in his ability to look back. If will Tom? get much and offer. It is just like a business. 
T=0.5: border Thats getting great great to the United States is not a believer agent, has already done in the bipartisan support and their efforts. to destroy our families and we can easily one 
T=1.0: border Thats getting great great for our Country! VOTE welcome down to the drive or after I wish everyone can the Pandemic, of our record economy, maybe amazing ever be the biggest political 
T=1.2: border Thats getting great little about jobs. this African-American Lunacy! on! Nice! True! VOTE! elected Thanks! exactly... be Presedent! in the attack on the Drive Look forward The being fat-not people A instead? 
Epoch 20/20
318/318 [==============================] - 15s 47ms/step - loss: 0.1651 - accuracy: 0.9515
----- Generating text after Epoch: 19 -----
T=0.2: can stop him. Danny will think well again! If we will be treating for their Democrat Party led the way to despite the name for a very important meeting, are are and Corrupt 
T=0.5: can stop him. Danny will think well again! If we will be treating at my vote on at Crime, Strong on Crime, tough if Biden Strong our great and beloved Brigitte U.S. I 
T=1.0: can stop him. Danny will Women be visiting and their scripts family. Plant his makes as putting to our Southern and will have amazing them. years be read oil thr worth got shellacked, 
T=1.2: can stop him. Danny will Must be interviewed by Refreshing at 1000 P.M. He or ring banned will wants place stay years Trump. at some things in the FBI and give practice their 
```
Here is the results of `model.summary()`:
```
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
bidirectional (Bidirectional (None, 256)               38353152  
_________________________________________________________________
dense (Dense)                (None, 49809)             12800913  
=================================================================
Total params: 51,154,065
Trainable params: 51,154,065
Non-trainable params: 0
_________________________________________________________________

```

The final weights used can be found at `weights-words.hdf5`

# Results (Characters)

While the character model didn't have the same high accuracy as words got to, it was still pretty impressive how lower temperatures produced realistic text sequences, and higher temperatures (mostly) were all English words.

In addition to not producing words (and just making characters) occasionally, another thing to notice was if the seed text ended with multiple `!`, the output would just be more `!`. 

Here is the plot from training:

<img src="words-results.png">

Here are the results from training:

```
Epoch 1/20
20238/20238 [==============================] - 137s 7ms/step - loss: 1.8959 - accuracy: 0.4669
----- Generating text after Epoch: 0 -----
T=0.2: Finally get to watch one of my favorite of the proods to the world and the process and the people of the the best and the prood will be the 
T=0.5: Finally get to watch one of my favorite the for any our country!!!!! The American Polition of I am the President Trump and well on the Unite
T=1.0: Finally get to watch one of my favorite intenest. We will be look if New York of others. At in the botal , new McCA and they late to especia
T=1.2: Finally get to watch one of my favorite our mentis. I hits potulate, no xic concerning Vote, go things!!!!!.Got quiting. Puilt, my voselle, 
Epoch 2/20
20238/20238 [==============================] - 135s 7ms/step - loss: 1.4506 - accuracy: 0.5825
----- Generating text after Epoch: 1 -----
T=0.2: I have arrived in Scotland and will be a great State of the U.S. and the Fake News Media is a great job and the Congress and State of the Wa
T=0.5: I have arrived in Scotland and will be a great State of the economy, WALL FRAU FIRE SEALL. We will be them and want the Senate Security and 
T=1.0: I have arrived in Scotland and will be a rocifer thanks foreage is Disant! Thank you this said back and American puntingly, need with but po
T=1.2: I have arrived in Scotland and will be a loopion changew abobeing, needs they largess. A will bvested mysolvadion hapance score or terwS Dir
Epoch 3/20
20238/20238 [==============================] - 136s 7ms/step - loss: 1.4038 - accuracy: 0.5945
----- Generating text after Epoch: 2 -----
T=0.2:  head to the Palmetto State Dont miss it will be the real with the problems and the respect out the process of the people of the U.S. and th
T=0.5:  head to the Palmetto State Dont miss it and so the news stand the part of the White House to the people of the packing the people of the pe
T=1.0:  head to the Palmetto State Dont miss it with interests at the Wark Its. He said it is we are railed Opportitions that I love honos the year
T=1.2:  head to the Palmetto State Dont miss it yearbyians if right. OTRING BEIOCEM Lincencs in already rigged.!!!!!! Of it well day. Good your boo
Epoch 4/20
20238/20238 [==============================] - 134s 7ms/step - loss: 1.3839 - accuracy: 0.5996
----- Generating text after Epoch: 3 -----
T=0.2: Why do all u haters follow Trump...pathed and the United States in the United States in the U.S. is a complete and the U.S. is a complete an
T=0.5: Why do all u haters follow Trump...pathed and the truly out and the Great Plecting with the 2016 Election. It is a fantastic Court is a coun
T=1.0: Why do all u haters follow Trump...pathed, first a throkem. Never Mexico is to tells that so much the Aldence and really reclacembilld can b
T=1.2: Why do all u haters follow Trump...pathed effective to have been fixed out of two Bred HELORRAWLY. Thx arms, she dare, adviden everyone? Imp
Epoch 5/20
20238/20238 [==============================] - 136s 7ms/step - loss: 1.3701 - accuracy: 0.6029
----- Generating text after Epoch: 4 -----
T=0.2: get my for President of the Unites State of The Committee and the American Party and the Democrats and the world the Democrat Part of the Un
T=0.5: get my for President of the Unites State Defense the Prime Minister will be with the best against me the State of the Senate and the Signet 
T=1.0: get my for President of the Unites State. It was you something Gerraagant poll and Prime Minister, Advatuin Obama, pleat that we kfew again,
T=1.2: get my for President of the Unites State,! He dont want people! This clowy I will ples a bad going to war State in Ded that Sechaties theres
Epoch 6/20
20238/20238 [==============================] - 137s 7ms/step - loss: 1.3630 - accuracy: 0.6049
----- Generating text after Epoch: 5 -----
T=0.2: Thank you, New Hampshire!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=0.5: Thank you, New Hampshire!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=1.0: Thank you, New Hampshire!!!!...!!!!!!!!!!!!!!.!!!!?m!!!.!J!!!!!!!!!.e.!!!!!.!!g!!!!!!!!?u!.l.te!t ..e!!!!!!!R!!!!!!6!!!!!!!!!
T=1.2: Thank you, New Hampshire!!!H.!VR!O.!!6!e.ti!!!6.-!4!!?!!!.?!s!n!,!!!!!!!S.!.0!?v-tkot!a!yar.,!ekdd-!!?m!d!Rut!D!.!!6n,!-!t!!!
Epoch 7/20
20238/20238 [==============================] - 137s 7ms/step - loss: 1.3576 - accuracy: 0.6065
----- Generating text after Epoch: 6 -----
T=0.2: Our Testing is the BEST in the World, by the U.S. is a great and state in the Trump Tariffs and the Wall in the U.S. and a strong on a great
T=0.5: Our Testing is the BEST in the World, by the people of Trump Administrations and that you are respected to do a trump that was being a massi
T=1.0: Our Testing is the BEST in the World, by a Counter of the Border, Aydin Economing The Obbary.better, anbeledulately come! Right Tax On Ameri
T=1.2: Our Testing is the BEST in the World, by 200rs year.........afual Prapternate. Thank you Depro, Anchormen who shemed in SC time for has miss
Epoch 8/20
20238/20238 [==============================] - 131s 6ms/step - loss: 1.3528 - accuracy: 0.6074
----- Generating text after Epoch: 7 -----
T=0.2: Ratings for NFL football are way down exceptions of the Democrats they will be a complete and we will be a form the people of the World and 
T=0.5: Ratings for NFL football are way down excited that its change. The Democrats are a great the Fake News They were a largely a very sack of th
T=1.0: Ratings for NFL football are way down excele byt communities well of the Vees America that Im Soce of China DiPLEALERS A mes. Milleornified 
T=1.2: Ratings for NFL football are way down expoce ir Ital Openly Russia will be outugt! -- safes billSous killed to destroy! Thank you now back-t
Epoch 9/20
20238/20238 [==============================] - 134s 7ms/step - loss: 1.3487 - accuracy: 0.6078
----- Generating text after Epoch: 8 -----
T=0.2: HEADLINES Donald Trump maintains huge leaders to the same to the new the people who was the people who can the Fake News Media will be the p
T=0.5: HEADLINES Donald Trump maintains huge leaders and action and the fact We will be a forgotten new work of the deal over men. Now they have a 
T=1.0: HEADLINES Donald Trump maintains huge leader and wifles of larger and afraid , being doing phone. As correctable. He should not know so incr
T=1.2: HEADLINES Donald Trump maintains huge less and plus to made who should the Opposit.,rack China doubt grabine, Freadal and pay absart like , 
Epoch 10/20
20238/20238 [==============================] - 133s 7ms/step - loss: 1.3459 - accuracy: 0.6092
----- Generating text after Epoch: 9 -----
T=0.2: President Trump just sounds right! Thank you! The Fake News Media will be a total leaders. They want to see you to make the only one the wor
T=0.5: President Trump just sounds right! Thank you! They dont have a states, the Fake News Media will be going to remained the Ireland who were do
T=1.0: President Trump just sounds right! Thanks! Strong! Herist.......and stund the up anger the treaton of EBOWIG spent bright with the country w
T=1.2: President Trump just sounds right! Thank you! Never again ail Virlation State is Walkers, when Dishoner!!!! Plus Artorm cared-who have back 
Epoch 11/20
20238/20238 [==============================] - 132s 7ms/step - loss: 1.3443 - accuracy: 0.6091
----- Generating text after Epoch: 10 -----
T=0.2: Natalie Harp, fighting Stage 2 Cancer and the Democrats and the U.S. is a bad and the U.S. is a total mess of the U.S. is the production to 
T=0.5: Natalie Harp, fighting Stage 2 Cancer and I will be fine beating of the Report is a androw Hillary as the Democrats want to be the Radical L
T=1.0: Natalie Harp, fighting Stage 2 Cancer and the United Nation. Unn Consersafing flights. Hopefulte in many economy, me. Special Rigged Clinton
T=1.2: Natalie Harp, fighting Stage 2 Cancer and make injogi-sy too copperso. Then Groht Hes enforcon, evidence that no brain unmats it.. they get 
Epoch 12/20
20238/20238 [==============================] - 135s 7ms/step - loss: 1.3452 - accuracy: 0.6091
----- Generating text after Epoch: 11 -----
T=0.2: Great poll in Iowa, where I just landed to the USA and the U.S. and the United States and the Democrats and the U.S. Many of the U.S. is the
T=0.5: Great poll in Iowa, where I just landed by the Military and the Fake News Mr. Trump and he was now the back interesting and looks lost of Co
T=1.0: Great poll in Iowa, where I just landed his Fillswere ever destroyed vessions arrey to 3.5 Minneone are went to me! Make America Himperson C
T=1.2: Great poll in Iowa, where I just landed dialogue help our very wained anything!!!!!!! He will have ever half no very forever guy for prosecu
Epoch 13/20
20238/20238 [==============================] - 132s 7ms/step - loss: 1.3408 - accuracy: 0.6101
----- Generating text after Epoch: 12 -----
T=0.2: The incompetence of our current administration in the U.S. State of American President Trump is a strong on the U.S. State of the U.S. State
T=0.5: The incompetence of our current administration and strong on Congress and the Democrats and the U.S. Governor of Tuest said the time and the
T=1.0: The incompetence of our current administration. Thank you GREAT EMERS - K.M.M. Speech we have been the great man! Expected to have the tarm 
T=1.2: The incompetence of our current administration, waste who refuse to Midstands that ashe Chuers. Just mis loagane with candivating a!This. Th
Epoch 14/20
20238/20238 [==============================] - 133s 7ms/step - loss: 1.3402 - accuracy: 0.6103
----- Generating text after Epoch: 13 -----
T=0.2: Why would anyone in Florida vote for lightwore and the words that the most supporters and the politician and the big statement and the Democ
T=0.5: Why would anyone in Florida vote for lightw all of the first fast to me in the country will be before and the courte to the words with the n
T=1.0: Why would anyone in Florida vote for ligtt my wifor court and betuser the Demacrative Country greatly even doing in a child Donald J. Trump 
T=1.2: Why would anyone in Florida vote for light at 35000.5 COB, Doomi and 8.AMIHIS akes asyauge? MSIR nation! Who ballots are campaign-is totally
Epoch 15/20
20238/20238 [==============================] - 135s 7ms/step - loss: 1.3396 - accuracy: 0.6104
----- Generating text after Epoch: 14 -----
T=0.2: For pres!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=0.5: For pres!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=1.0: For prespe.y!!!!!! !.!!?!p6yyh!t.l?!.!y!.u, !!!!!!.!!!!!..!a..srnT!!!.!k!!!!!!!!!!.!!sy!!!!!!!..!!.-.s..!e.!
T=1.2: For pres. !ad!.w!!!!ro!!n!.ad.try-e!a?!!!ya.!u!!!s.a-.!!M!ve5apyeu.!.pw!!!U!-!6.b.ad!!fr..t!sapne.!e!dn!!!a!
Epoch 16/20
20238/20238 [==============================] - 132s 7ms/step - loss: 1.3396 - accuracy: 0.6102
----- Generating text after Epoch: 15 -----
T=0.2: Our great Economy is the talk of everyone will be a real for the best of the USA! The Democrats and the best energy and we can be a great co
T=0.5: Our great Economy is the talk of everyone in the Border, he said that the votes and the Fake News MAKE AMERICA GREAT AGAIN! The U.S. Golf co
T=1.0: Our great Economy is the talk of everyone , they are bs on to be fewer absolutely better to HARNE!! Request Arrestry CoronaVirus of Coping B
T=1.2: Our great Economy is the talk of everyone world. FOX!J netion. Reherfel. Unbeter 136, teclearding. Ebova Speice. NOMERY MUST. False , its lo
Epoch 17/20
20238/20238 [==============================] - 130s 6ms/step - loss: 1.3388 - accuracy: 0.6104
----- Generating text after Epoch: 16 -----
T=0.2: is unfit to serve.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=0.5: is unfit to serve.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
T=1.0: is unfit to serve.ad.!!!J!!!!! ,!!!!!!!!-!..r,!!!!!!!!W!u!..!!!!!,!!Y!!!!!!!da!.!!!!!6!.!.!d!!!!!!!!!!!!!!!!!!!!F!t!!-
T=1.2: is unfit to serve.uytet,!t!!!Yi,.o!?!t!.!ea!!rt!..!!!!SY!-!!!!!S!-.r.!.e!!.rlntaw.!!.!!!!!.!!g!.!pyrla!!n!.,5!!s!!!!HY
Epoch 18/20
20238/20238 [==============================] - 130s 6ms/step - loss: 1.3375 - accuracy: 0.6107
----- Generating text after Epoch: 17 -----
T=0.2: I am thrilled to nominate Dr. as our next President Trump Taxes and the United States and the United States President of the United States a
T=0.5: I am thrilled to nominate Dr. as our next gleys with me a failed on The Great Schums Polls and the Trump. The last night to provide the Fake
T=1.0: I am thrilled to nominate Dr. as our next yesterday, he would make our voting on 147 MECATER?Ye the World alaval havent. So while thing with
T=1.2: I am thrilled to nominate Dr. as our next, after Russiasure pububl rid in a by all care of Illassy violly. Bad Twaild Prior2 on Subvecs time
Epoch 19/20
20238/20238 [==============================] - 130s 6ms/step - loss: 1.3366 - accuracy: 0.6111
----- Generating text after Epoch: 18 -----
T=0.2: I love the Mexican people, but Mexico is a companies to see the problems, the White House to the U.S. and the U.S. Security will be a tough 
T=0.5: I love the Mexican people, but Mexico is the most shot to make America Great Again! I would be because the American Democrat Government Scam
T=1.0: I love the Mexican people, but Mexico is while polls. Lewes and in Collusion Cruz. China? Masy make America Great Again! Evidence out thing 
T=1.2: I love the Mexican people, but Mexico is Steve what will become Presidential firm. Not would do the voter in Mikes-in minute negace bill in!
Epoch 20/20
20238/20238 [==============================] - 138s 7ms/step - loss: 1.3384 - accuracy: 0.6106
----- Generating text after Epoch: 19 -----
T=0.2: is really starting to campaign hard for the world of the United States to the Fake News Media and the Fake News Make America Great Again! Th
T=0.5: is really starting to campaign hard for Washington Democrats that it is not needed for his guy to make America Great Again! And they want to
T=1.0: is really starting to campaign hard for back man and work, There was NO COME!! Chiefson and showed these Fake News. Biden than our, discrapi
T=1.2: is really starting to campaign hard for report. Mi Secrings! Runs......with driffed number farmers, I am noterful 1 ubwind. Tues action. The


```

Here are the results of `model.summary()`:
```
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
gru_2 (GRU)                  (None, 128)               76800     
_________________________________________________________________
dense_2 (Dense)              (None, 70)                9030      
=================================================================
Total params: 85,830
Trainable params: 85,830
Non-trainable params: 0
_________________________________________________________________
```

The final weights used can be found at `weights-chars.hdf5` (much smaller than words)

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               76800     
_________________________________________________________________
dense (Dense)                (None, 70)                9030      
Total params: 85,830
Trainable params: 85,830
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# code to generate the graph
try:
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 22})


    fig, ax1 = plt.subplots(figsize=(20,15))
    fig.patch.set_facecolor('#FFFFFF')
    ax2 = ax1.twinx()
    ax1.plot(history.history['loss'], color='red')
    ax2.plot(history.history['accuracy'], color='blue')

    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss', color='red')
    ax2.set_ylabel('Accuracy', color='blue')


    ax1.set_title('Training Metrics (3 words)')
    plt.show()
except Exception as e: # training didn't finish, so no history
    pass



In [ ]:
# model.save_weights('weights-backup.hdf5')

In [ ]:
# TensorBoard results (graph, epochs, etc)
%reload_ext tensorboard
%tensorboard --logdir logs